In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random 
import tensorflow as tf
import datetime
import torchvision
from torchvision import datasets, transforms

In [ ]:
#load dataset
import kagglehub
# Download latest version
path = kagglehub.dataset_download("ronakp004/autism-spectrum-detection-from-kaggle-zenodo")
PATH_TO_DATASET = path + '/Cleaned Dataset'

Path to dataset files: /home/grimm/.cache/kagglehub/datasets/ronakp004/autism-spectrum-detection-from-kaggle-zenodo/versions/11


In [14]:
#set seed
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
tf.random.set_seed(0)

In [26]:
BATCH_SIZE = 32
NUM_WORKERS = 4
IMAGE_SIZE = 224
EMBEDDING_SIZE = 512

In [19]:
import os
from torchvision.io import decode_image
from sklearn.preprocessing import LabelEncoder

class AutismDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, transform=None, label_encoder=None, fit_label_encoder=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
        if label_encoder is None:
            self.label_encoder = LabelEncoder()
            if fit_label_encoder:
                self.labels = self.label_encoder.fit_transform(self.data['labels'])
            else:
                raise ValueError("Must provide label_encoder or set fit_label_encoder=True")
        else:
            self.label_encoder = label_encoder
            self.labels = self.label_encoder.transform(self.data['labels'])
        
        self.image_paths = self.data['image_path'].tolist()
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Load image
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            image = Image.new('RGB', (IMAGE_SIZE, IMAGE_SIZE), color='black')
        
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.long)

In [27]:
# Conservative augmentation for face recognition models
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.3),  # Reduced probability
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.02),  # Very gentle
    transforms.RandomRotation(degrees=3),  # Minimal rotation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = AutismDataset(csv_file=os.path.join(PATH_TO_DATASET, 'cleaned_train.csv'),
                               transform=train_transform,
                               fit_label_encoder=True)

val_dataset = AutismDataset(csv_file=os.path.join(PATH_TO_DATASET, 'cleaned_valid.csv'),
                             transform=val_test_transform,
                             label_encoder=train_dataset.label_encoder)
test_dataset = AutismDataset(csv_file=os.path.join(PATH_TO_DATASET, 'cleaned_test.csv'),
                              transform=val_test_transform,
                              label_encoder=train_dataset.label_encoder)

In [24]:
train_dataset

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()



writer.add_image('images', grid, 0)
writer.add_graph(model, images)
writer.close()

In [11]:
writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [10]:
import os
os.getcwd()

'/home/grimm/Documents/Projects/Python/HSSE-CV/seminars'